<a href="https://colab.research.google.com/github/shreyammb/INNOV8TIGERS/blob/main/predict_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio==1.3.9 numpy pandas scikit-image scikit-learn xgboost lightgbm joblib tqdm shapely

!pip install numpy==1.26.4
import numpy as np
print("NumPy version:", np.__version__)

In [ ]:
import os, math, json
import numpy as np, pandas as pd
import rasterio
from rasterio.warp import reproject, Resampling as WarpResampling
from skimage.feature import local_binary_pattern, canny
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, opening, closing, square
from scipy.ndimage import uniform_filter
from skimage.util import img_as_ubyte
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xgb, lightgbm as lgb
from joblib import Parallel, delayed
from tqdm import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

BASE_TIF = "/content/drive/MyDrive/HYD/S2_BOM_V2.tif"
MASK_TIF = "/content/drive/MyDrive/HYD/sample_mask.tif"

OUT_DIR = "/content/drive/MyDrive/HYD/out_slum"
os.makedirs(OUT_DIR, exist_ok=True)

print("Base:", BASE_TIF)
print("Mask:", MASK_TIF)
print("Out dir:", OUT_DIR)

LBP_RADII = [1, 3]
EDGE_WINDOWS = [3, 7]
ROOF_WINDOWS = [3, 7]
LACUNARITY_WINDOWS = [9, 21]
RANK_GLCM_WINDOW = 7

RF_PARAMS = {"n_estimators":200, "max_depth":12, "class_weight":"balanced", "random_state":42}
XGB_PARAMS = {"n_estimators":300, "max_depth":4, "learning_rate":0.05, "eval_metric":"logloss"}
LGB_PARAMS = {"n_estimators":500, "num_leaves":31, "learning_rate":0.05}

In [ ]:
def read_raster_meta(path):
    with rasterio.open(path) as src:
        names = [src.descriptions[i] or f"Band_{i+1}" for i in range(src.count)]
        meta = src.meta.copy()
    return names, meta

def read_full_raster(path):
    with rasterio.open(path) as src:
        arr = src.read().astype(np.float32)
        names = [src.descriptions[i] or f"Band_{i+1}" for i in range(src.count)]
        meta = src.meta.copy()
    return arr, names, meta

def find_idx_by_sub(substr):
    s=substr.upper()
    for i,nm in enumerate(names):
        if s in nm.upper():
            return i
    return None